https://www.sfu.ca/~ssurjano/holder.html
https://www.sfu.ca/~ssurjano/crossit.html

In [52]:
import numpy as np

from typing import List, Tuple

In [53]:
population_size = 100
bits_per_input = 8

In [54]:
def cross_in_tray(x1: int, x2: int) -> int:
    return -0.0001 * \
        (abs(abs(np.sin(x1) * np.sin(x2) * np.exp (100 - (np.sqrt(x1 ** 2 + x2 ** 2)) / np.pi) \
    )) + 1) ** 0.1

In [55]:
def binary_to_float(binary: str, min_value: int, max_value: int, interval_size: int = 20) -> float:
    decimal = int(binary, 2)

    return min_value + decimal / max_value * (2 ** interval_size - 1)

In [56]:
def float_to_binary(real: str, min_value: int, max_value: int, interval_size: int = 20) -> str:
    decimal = int((real - min_value) * (2 ** interval_size - 1)) / (max_value - min_value)

    binary = format(decimal, f'0{interval_size}b')
    return binary

In [57]:
def generate_individual():
    individual = ''.join(np.random.choice(['0', '1']) for _ in range(bits_per_input * 2))

    return individual

In [58]:
def decode(individual: str) -> Tuple[float, float]:
    x1_bin = individual[bits_per_input:]
    x2_bin = individual[:bits_per_input]

    x1 = binary_to_float(x1_bin)
    x2 = binary_to_float(x2_bin)

    return x1, x2

In [59]:
def fitness(individual: str) -> float:
    x1, x2 = decode(individual)

    return -1 * cross_in_tray(x1, x2)

In [61]:
def generate_initial_population(population_size) -> List[str]:
    generation = []
    
    for _ in population_size:
        individual = generate_individual()
        fitness_score = fitness(individual)

        generation.push({"binary": individual, "score": fitness_score})

    return generation